                                            IMPORTO LAS LIBRERIAS

In [57]:
#importo las librerias
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
import glob
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

                                            CARGA DE LOS DATASET

In [100]:
#cargo los datasets
venta = pd.read_csv('../Datasets/Venta.csv')
gasto = pd.read_csv('../Datasets/Gasto.csv')
compra = pd.read_csv('../Datasets/Compra.csv')
localidades = pd.read_csv('../Datasets/Localidades.csv')
proveedores = pd.read_csv('../Datasets/Proveedores.csv', encoding='ansi')
sucursales = pd.read_csv('../Datasets/Sucursales.csv', delimiter=';')
clientes = pd.read_csv('../Datasets/Clientes.csv',encoding = "utf-8",delimiter = ';', usecols=["ID","Provincia","Nombre_y_Apellido","Domicilio","Telefono","Edad","Localidad","X","Y","col10"])
clientes2 = pd.read_csv('../Datasets/Clientes_v2.csv', delimiter=';')
clientes_dic2020 = pd.read_csv('../Datasets/Clientes_Dic2020.csv', delimiter=';')

# NORMALIZACION DE TABLA VENTA

ANALISIS DE LOS DATOS DE LA TABLA:

In [59]:
#hago una copia del dataframe original para trabajar con una copia
venta2 = venta.copy()

In [60]:
#visualizo la tabla venta
venta2.head()

,IdVenta,Fecha,Fecha_Entrega,IdCanal,IdCliente,IdSucursal,IdEmpleado,IdProducto,Precio,Cantidad
0,1,2018-03-09,2018-03-17,3,969,13,1674,42817,813.12,2.0
1,2,2018-12-28,2018-12-29,2,884,13,1674,42795,543.18,3.0
2,3,2016-03-28,2016-03-31,2,1722,13,1674,42837,430.32,1.0
3,4,2017-10-23,2017-10-24,3,2876,13,1674,42834,818.84,2.0
4,5,2017-11-22,2017-11-25,2,678,13,1674,42825,554.18,3.0


In [61]:
#visualizo el tipo de datos, numero de col y filas y los valores nulos
venta2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46180 entries, 0 to 46179
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   IdVenta        46180 non-null  int64  
 1   Fecha          46180 non-null  object 
 2   Fecha_Entrega  46180 non-null  object 
 3   IdCanal        46180 non-null  int64  
 4   IdCliente      46180 non-null  int64  
 5   IdSucursal     46180 non-null  int64  
 6   IdEmpleado     46180 non-null  int64  
 7   IdProducto     46180 non-null  int64  
 8   Precio         45260 non-null  float64
 9   Cantidad       45296 non-null  float64
dtypes: float64(2), int64(6), object(2)
memory usage: 3.5+ MB


In [62]:
#sumo los valores nulos por columna
venta2.isnull().sum()

IdVenta            0
Fecha              0
Fecha_Entrega      0
IdCanal            0
IdCliente          0
IdSucursal         0
IdEmpleado         0
IdProducto         0
Precio           920
Cantidad         884
dtype: int64

TRATAMIENTO DE OUTLIERS EN LA COLUMNA PRECIO:

Cálculo de outliers 

In [63]:
#Tratamiento de Outliers
Q1 = venta2["Precio"].quantile(0.25)
Q3 = venta2["Precio"].quantile(0.75)
#Se calcula el rango intercuartilico IQR.
IQR = Q3 -Q1
outliers_prec = (Q3 + (1.5*IQR))
#genero un nuevo dataframe pero sin los valores outliers
ventaSinOut = venta2[venta2['Precio']<outliers_prec]
outliers_prec #a partir de 3815,5 en adelante son outliers


3815.5

Creo una columna outliers_precio para identidicar los outliers con cero

In [64]:
#agrego una col nueva con valores de 1
venta2['outliers_precio'] = np.ones(venta2.shape[0])
#coloco un cero donde el valor de la col Precio es mayor al valor de outlier para identificar los ouliers con cero
venta2.loc[(venta2['Precio'] >= outliers_prec),"outliers_precio"] = 0

Genero una nueva columna Precio2 con los valores agrupados por IdProducto de mi nuevo df de ventas sin outliers

In [65]:
#agrupo por idproducto para hallar el valor medio de cada producto 
Precio_Producto = ventaSinOut.groupby(['IdProducto'])['Precio'].mean().reset_index()
#renombro la columna precio de mi nuevo df
Precio_Producto = Precio_Producto.rename(columns= {"Precio":"Precio2"})
#úno a travez de IdProducto la tabla venta2 con Precio_producto para agregar la col Precio2
venta2 = pd.merge(venta2,Precio_Producto, how="left", on=["IdProducto"])
#ordeno los valores de IdVenta en forma desc
#venta2.sort_values("IdVenta")
venta2.head()

,IdVenta,Fecha,Fecha_Entrega,IdCanal,IdCliente,IdSucursal,IdEmpleado,IdProducto,Precio,Cantidad,outliers_precio,Precio2
0,1,2018-03-09,2018-03-17,3,969,13,1674,42817,813.12,2.0,1.0,813.12
1,2,2018-12-28,2018-12-29,2,884,13,1674,42795,543.18,3.0,1.0,543.18
2,3,2016-03-28,2016-03-31,2,1722,13,1674,42837,430.32,1.0,1.0,430.32
3,4,2017-10-23,2017-10-24,3,2876,13,1674,42834,818.84,2.0,1.0,818.84
4,5,2017-11-22,2017-11-25,2,678,13,1674,42825,554.18,3.0,1.0,554.18


In [66]:
#inserto en los campos vacios de Precio2 el valor promedio de los precios sin outliers
venta2.Precio2.fillna(round(ventaSinOut.Precio.mean(),2),inplace=True)

In [67]:
#le coloco un cero a todos los valores nulos de la col Precio
venta2.Precio.fillna(0, inplace=True)

In [68]:
venta2.head()

,IdVenta,Fecha,Fecha_Entrega,IdCanal,IdCliente,IdSucursal,IdEmpleado,IdProducto,Precio,Cantidad,outliers_precio,Precio2
0,1,2018-03-09,2018-03-17,3,969,13,1674,42817,813.12,2.0,1.0,813.12
1,2,2018-12-28,2018-12-29,2,884,13,1674,42795,543.18,3.0,1.0,543.18
2,3,2016-03-28,2016-03-31,2,1722,13,1674,42837,430.32,1.0,1.0,430.32
3,4,2017-10-23,2017-10-24,3,2876,13,1674,42834,818.84,2.0,1.0,818.84
4,5,2017-11-22,2017-11-25,2,678,13,1674,42825,554.18,3.0,1.0,554.18


Inserto los valores faltantes en la columna Precio

In [69]:
#con un bucle for cambio los valores en la col Precio que son igual a cero o son outliers y les coloco
#el valor de la col precio2
#for i in range(0,venta2.shape[0]):
  #if venta2.Precio[i] == 0 or venta2.Precio[i] >outliers_prec :
    #venta2.Precio[i] = venta2.Precio2[i]

for i in range(0,venta2.shape[0]):
  if venta2.Precio[i] == 0 or venta2.Precio[i]>outliers_prec:
    venta2.iloc[i,8]=venta2.iloc[i,11] # 8 es precio, 11 es precio 2

In [70]:
#elimino la col Precio2
venta2.drop(columns='Precio2', inplace= True)

In [71]:
# Graficar Ventas por Año
venta2['Total'] = venta2['Precio'] * venta2['Cantidad']
sns.lineplot(data=venta2.groupby(venta2.Fecha.dt.year).sum(), x="Fecha", y="Total").set_title('Evolución de Ventas por Año', fontsize=20)

AttributeError: Can only use .dt accessor with datetimelike values

TRATAMIENTO DE OULIERS COLUMNA CANTIDAD:

Cálculo de outliers

In [449]:
#Tratamiento de Outliers columna cantidad
Q1 = venta2["Cantidad"].quantile(0.25)
Q3 = venta2["Cantidad"].quantile(0.75)
#Se calcula el rango intercuartilico IQR.
IQR = Q3 -Q1
outliers_cant = (Q3 + (1.5*IQR)) 
ventaCantSinOut = venta2[venta2["Cantidad"]<outliers_cant]
outliers_cant #son outliers a partir de 4.5 

4.5

Creo una columna outliers_cant para identificar los outliers con cero

In [450]:
#creo una columna outliers_cant con unos
venta2['outliers_cant'] = np.ones(venta2.shape[0])
#coloco un cero en la col outliers_cant donde cantidad es mayor o igual a outliers_canti
venta2.loc[(venta2['Cantidad'] >= outliers_cant), 'outliers_cant'] = 0

Coloco el promedio de cantidad de ventas sin outliers en los outliers de cantidad

In [451]:
#saco el promerdio de cantidades de cantiad de ventas sin outliers
cantidadMedia = round(ventaCantSinOut.Cantidad.mean())
#coloco el valor promedio en cantidad donde cantidad es outlier
venta2.loc[(venta2['Cantidad'] >= outliers_cant),"Cantidad"] = cantidadMedia #2

In [452]:
#coloco un 1 en los valores nulos de la columna Cantidad
venta2.Cantidad.fillna(1, inplace=True)

Creo una columna Monto para visualizar el monto de cada venta

In [453]:
#creo una nueva col con el monto de cada registo de la tabla
venta2['Monto'] = venta2['Precio']*venta2['Cantidad']

# NORMALIZACION DE LA TABLA COMPRA

VISUALIZACION DE LOS DATOS:

In [454]:
#genero una copia del dataframe 
compra2 = compra.copy()

In [455]:
compra2.head()

,IdCompra,Fecha,Fecha_Año,Fecha_Mes,Fecha_Periodo,IdProducto,Cantidad,Precio,IdProveedor
0,1,1/30/2015,2015,1,201501,42832,13,560.51,12
1,2,1/30/2015,2015,1,201501,42833,11,497.58,7
2,3,1/30/2015,2015,1,201501,42834,1,588.50,6
3,4,1/30/2015,2015,1,201501,42835,9,567.66,14
4,5,1/30/2015,2015,1,201501,42839,14,231.31,2


In [456]:
compra2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11539 entries, 0 to 11538
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   IdCompra       11539 non-null  int64  
 1   Fecha          11539 non-null  object 
 2   Fecha_Año      11539 non-null  int64  
 3   Fecha_Mes      11539 non-null  int64  
 4   Fecha_Periodo  11539 non-null  int64  
 5   IdProducto     11539 non-null  int64  
 6   Cantidad       11539 non-null  int64  
 7   Precio         11172 non-null  float64
 8   IdProveedor    11539 non-null  int64  
dtypes: float64(1), int64(7), object(1)
memory usage: 811.5+ KB


In [457]:
compra2.isnull().sum()

IdCompra           0
Fecha              0
Fecha_Año          0
Fecha_Mes          0
Fecha_Periodo      0
IdProducto         0
Cantidad           0
Precio           367
IdProveedor        0
dtype: int64

In [458]:
compra2.describe().T

,count,mean,std,min,25%,50%,75%,max
IdCompra,11539.0,5770.000000,3331.166713,1.00,2885.50,5770.000,8654.5000,11539.0
Fecha_Año,11539.0,2017.671549,1.770288,2015.00,2016.00,2018.000,2019.0000,2020.0
Fecha_Mes,11539.0,4.570500,3.168731,1.00,1.00,4.000,7.0000,12.0
Fecha_Periodo,11539.0,201771.725366,177.830326,201501.00,201605.00,201801.000,201908.0000,202012.0
IdProducto,11539.0,42900.634890,83.539532,42737.00,42832.00,42902.000,42973.0000,43043.0
Cantidad,11539.0,8.826415,6.412074,1.00,4.00,7.000,12.0000,54.0
Precio,11172.0,3613.646813,100663.022573,1.86,306.83,463.485,1260.3225,10445800.0
IdProveedor,11539.0,7.533928,3.770609,1.00,4.00,8.000,11.0000,14.0


TRATAMIENTO DE OUTLIERS DE LA COLUMNA PRECIO

Deteccion de outliers 

In [459]:
#Tratamiento de Outliers
Q1 = compra2["Precio"].quantile(0.25)
Q3 = compra2["Precio"].quantile(0.75)
#Se calcula el rango intercuartilico IQR.
IQR = Q3 -Q1
outliers_prec = (Q3 + (1.5*IQR)) 
#genero un nuevo dataframe pero sin los valores outliers
compra2SinOut = compra2[compra2["Precio"]<outliers_prec]
outliers_prec #a partir de 2690.56 en adelante son outliers

2690.5612499999997

Creo una columna outliers_prec para identificar los outliers con un cero

In [460]:
compra2['Outliers_prec'] = np.ones(compra2.shape[0])
compra2.loc[(compra2['Precio']>= outliers_prec), 'Outliers_prec'] = 0

Genero una nueva columna Precio2 con los valores agrupados por IdProducto de mi nuevo df de ventas sin outliers

In [461]:
#agrupo por idProducto en la tabla sin outliers y le saco el valor del promedio de cada producto
Precio_Producto= compra2.groupby(["IdProducto"])["Precio"].mean().reset_index()
#renomnbro la columna precio por precio2 en el nuevo dataframe
Precio_Producto= Precio_Producto.rename(columns= {"Precio":"Precio2"})

In [462]:
#hago una union a travez de IdProducto 
compra2 = pd.merge(compra2,Precio_Producto, how="left", on=["IdProducto"])
#ordeno los valores de IdVenta en forma desc
#compra2.sort_values("IdCompra")
compra2.head()

,IdCompra,Fecha,Fecha_Año,Fecha_Mes,Fecha_Periodo,IdProducto,Cantidad,Precio,IdProveedor,Outliers_prec,Precio2
0,1,1/30/2015,2015,1,201501,42832,13,560.51,12,1.0,2181.913636
1,2,1/30/2015,2015,1,201501,42833,11,497.58,7,1.0,3057.383261
2,3,1/30/2015,2015,1,201501,42834,1,588.50,6,1.0,1911.816596
3,4,1/30/2015,2015,1,201501,42835,9,567.66,14,1.0,2324.104118
4,5,1/30/2015,2015,1,201501,42839,14,231.31,2,1.0,250.286333


In [463]:
#inserto en los campos vacios de Precio2 el valor promedio de los precios sin outliers
compra2.Precio2.fillna(compra2SinOut.Precio.mean(),inplace=True)

In [464]:
#coloco un cero en los valores nulos de la columna precio
compra2.Precio.fillna(0, inplace=True)

In [465]:
compra2.head()

,IdCompra,Fecha,Fecha_Año,Fecha_Mes,Fecha_Periodo,IdProducto,Cantidad,Precio,IdProveedor,Outliers_prec,Precio2
0,1,1/30/2015,2015,1,201501,42832,13,560.51,12,1.0,2181.913636
1,2,1/30/2015,2015,1,201501,42833,11,497.58,7,1.0,3057.383261
2,3,1/30/2015,2015,1,201501,42834,1,588.50,6,1.0,1911.816596
3,4,1/30/2015,2015,1,201501,42835,9,567.66,14,1.0,2324.104118
4,5,1/30/2015,2015,1,201501,42839,14,231.31,2,1.0,250.286333


In [466]:
#coloco los valores faltantes y outliers de la columna precio usando los valores de la columna precio2
#for i in range(0,compra2.shape[0]):
    #if compra2.Precio[i] == 0 or compra2.Precio[i] > outliers_prec:
        #compra2.Precio[i] = compra2.Precio2[i]

#Este bucle reemplaza los ceros y outliers por los valores promedios 
for i in range(0,compra2.shape[0]):
    if compra2.Precio[i] == 0 or compra2.Precio[i]>outliers_prec:
        compra2.iloc[i,7]=compra2.iloc[i,10] # 7 es precio, 10 es precio 2

In [467]:
#elimino la columna 'Precio2'
compra2.drop(columns='Precio2', inplace= True)

TRATAMIENTO DE OUTLIERS DE LA COLUMNA CANTIDAD

Deteccion de outliers 

In [468]:
#Se calcula el rango intercuartilico IQR.
Q1 = compra2["Cantidad"].quantile(0.25)
Q3 = compra2["Cantidad"].quantile(0.75)
IQR = Q3 -Q1
outliersSup = (Q3 + (1.5*IQR)) 
# se crea un df sin outliers, se calcula la media de compras,y se imputan como valor en los casos que sean outliers.
compraCantSinOut = compra[compra2["Cantidad"]<outliersSup]
cantidadMedia = round(compraCantSinOut.Cantidad.mean())
compra2.loc[(compra2['Cantidad'] >= outliersSup),"Cantidad"] = cantidadMedia #8

Creo una columna para identificar los outliers con ceros

In [469]:
#creo una columna outliers y la completo con unos
compra2['Outliers_cant'] = np.ones(compra2.shape[0])
#coloco un cero donde el valor de la col Precio es mayor al valor de outlier para identificar los ouliers con cero
compra2["Outliers_cant"].loc[(compra2['Cantidad'] >= outliersSup)] = 0

C:\Users\Lea\AppData\Local\Temp\ipykernel_17532\3734470604.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compra2["Outliers_cant"].loc[(compra2['Cantidad'] >= outliersSup)] = 0


In [470]:
compra2['Total'] = compra2['Precio']*compra2['Cantidad']

In [471]:
compra2.head()

,IdCompra,Fecha,Fecha_Año,Fecha_Mes,Fecha_Periodo,IdProducto,Cantidad,Precio,IdProveedor,Outliers_prec,Outliers_cant,Total
0,1,1/30/2015,2015,1,201501,42832,13,560.51,12,1.0,1.0,7286.63
1,2,1/30/2015,2015,1,201501,42833,11,497.58,7,1.0,1.0,5473.38
2,3,1/30/2015,2015,1,201501,42834,1,588.50,6,1.0,1.0,588.50
3,4,1/30/2015,2015,1,201501,42835,9,567.66,14,1.0,1.0,5108.94
4,5,1/30/2015,2015,1,201501,42839,14,231.31,2,1.0,1.0,3238.34


# NORMALIZACION DE TABLA LOCALIDADES

VISUALIZACION DE DATOS:

In [358]:
localidades2 = localidades.copy()

In [359]:
localidades2.head()

,categoria,centroide_lat,centroide_lon,departamento_id,departamento_nombre,fuente,id,localidad_censal_id,localidad_censal_nombre,municipio_id,municipio_nombre,nombre,provincia_id,provincia_nombre
0,Localidad simple,-35.033073,-60.280620,6021.0,Alberti,INDEC,6021010000,6021010,Alberti,60021.0,Alberti,ALBERTI,6,Buenos Aires
1,Localidad simple,-34.868119,-60.393971,6021.0,Alberti,INDEC,6021020000,6021020,Coronel Seguí,60021.0,Alberti,CORONEL SEGUI,6,Buenos Aires
2,Componente de localidad compuesta,-35.068014,-60.402597,6021.0,Alberti,INDEC,6021030000,6021030,Mechita,60021.0,Alberti,MECHITA,6,Buenos Aires
3,Localidad simple,-35.124382,-60.220061,6021.0,Alberti,INDEC,6021040000,6021040,Pla,60021.0,Alberti,PLA,6,Buenos Aires
4,Localidad simple,-35.109618,-60.070551,6021.0,Alberti,INDEC,6021050000,6021050,Villa Grisolía,60021.0,Alberti,VILLA GRISOLIA,6,Buenos Aires


In [360]:
localidades2.isnull().sum()

categoria                    0
centroide_lat                0
centroide_lon                0
departamento_id              1
departamento_nombre          1
fuente                       0
id                           0
localidad_censal_id          0
localidad_censal_nombre      0
municipio_id               543
municipio_nombre           494
nombre                       0
provincia_id                 0
provincia_nombre             0
dtype: int64

In [361]:
localidades2["nombre"]=localidades2.apply(lambda x: "Ciudad de Buenos Aires" if x["municipio_nombre"]=="Capital Federal"  else x["nombre"], axis=1)

In [362]:
#renombro las columnas siguientes
localidades2 = localidades2.rename(columns={'centroide_lat':'Latitud','centroide_lon':'Longitud','departamento_id':'IdDepartamento','id':'IdLocalidad','nombre':'Localidad', 'provincia_id':'IdProvincia','provincia_nombre':'Provincia'})

In [363]:
localidades2.head()

,categoria,Latitud,Longitud,IdDepartamento,departamento_nombre,fuente,IdLocalidad,localidad_censal_id,localidad_censal_nombre,municipio_id,municipio_nombre,Localidad,IdProvincia,Provincia
0,Localidad simple,-35.033073,-60.280620,6021.0,Alberti,INDEC,6021010000,6021010,Alberti,60021.0,Alberti,ALBERTI,6,Buenos Aires
1,Localidad simple,-34.868119,-60.393971,6021.0,Alberti,INDEC,6021020000,6021020,Coronel Seguí,60021.0,Alberti,CORONEL SEGUI,6,Buenos Aires
2,Componente de localidad compuesta,-35.068014,-60.402597,6021.0,Alberti,INDEC,6021030000,6021030,Mechita,60021.0,Alberti,MECHITA,6,Buenos Aires
3,Localidad simple,-35.124382,-60.220061,6021.0,Alberti,INDEC,6021040000,6021040,Pla,60021.0,Alberti,PLA,6,Buenos Aires
4,Localidad simple,-35.109618,-60.070551,6021.0,Alberti,INDEC,6021050000,6021050,Villa Grisolía,60021.0,Alberti,VILLA GRISOLIA,6,Buenos Aires


In [364]:
#elimino las siguientes columnas
localidades2.drop(columns=(['categoria','fuente','departamento_nombre','localidad_censal_id','localidad_censal_nombre','municipio_id','municipio_nombre']), inplace=True)

In [365]:
#normalizo los datos tipo string de dichas columnas
localidades2["Localidad"]= localidades2["Localidad"].str.title()
localidades2["Provincia"]= localidades2["Provincia"].str.title()

In [366]:
#elimino los duplicados
localidades2.drop_duplicates(subset=['Localidad','Provincia'], inplace=True)

In [367]:
#reseteo los indices
localidades2.reset_index(inplace=True)

In [368]:
localidades2.head()

,index,Latitud,Longitud,IdDepartamento,IdLocalidad,Localidad,IdProvincia,Provincia
0,0,-35.033073,-60.280620,6021.0,6021010000,Alberti,6,Buenos Aires
1,1,-34.868119,-60.393971,6021.0,6021020000,Coronel Segui,6,Buenos Aires
2,2,-35.068014,-60.402597,6021.0,6021030000,Mechita,6,Buenos Aires
3,3,-35.124382,-60.220061,6021.0,6021040000,Pla,6,Buenos Aires
4,4,-35.109618,-60.070551,6021.0,6021050000,Villa Grisolia,6,Buenos Aires


In [369]:
localidades2["Localidad"]=[x.replace('á',"a") for x in localidades2["Localidad"]]
localidades2["Localidad"]=[x.replace('é',"e") for x in localidades2["Localidad"]]
localidades2["Localidad"]=[x.replace('í',"i") for x in localidades2["Localidad"]]
localidades2["Localidad"]=[x.replace('ó',"o") for x in localidades2["Localidad"]]
localidades2["Localidad"]=[x.replace('ú',"u") for x in localidades2["Localidad"]]
localidades2["Provincia"]=[x.replace('á',"a") for x in localidades2["Provincia"]]
localidades2["Provincia"]=[x.replace('é',"e") for x in localidades2["Provincia"]]
localidades2["Provincia"]=[x.replace('í',"i") for x in localidades2["Provincia"]]
localidades2["Provincia"]=[x.replace('ó',"o") for x in localidades2["Provincia"]]
localidades2["Provincia"]=[x.replace('ú',"u") for x in localidades2["Provincia"]]

In [370]:
lista_provincias = list(localidades2["Provincia"].unique())

In [371]:
localidades2.shape

(4001, 8)

# NORMALIZACION DE TABLA CLIENTE

In [372]:
clientes2 = clientes.copy()

In [373]:
clientes2.head()

,ID,Provincia,Nombre_y_Apellido,Domicilio,Telefono,Edad,Localidad,X,Y,col10
0,1,Buenos Aires,HEBER JONI SANTANA,LAS HERAS Y BAT. 24 DE FEBRERO 4150 RINCON DE...,42-5161,58,LOMA VERDE,"-58,81850307","-34,30997088",NaN
1,2,Buenos Aires,ANA SAPRIZA,PUEYRREDON Y DUPUY RUTA3 KM 52.500 S/N BÂº LO...,49-7578,61,SANTA ROSA,"-58,73073751","-34,93908311",NaN
2,3,Buenos Aires,FERNANDO LUIS SARALEGUI,CALDERON DE LA BARCA 498,49-3435,15,TORRES,"-59,12794068","-34,43082199",NaN
3,4,Buenos Aires,MANUELA SARASOLA,"RUTA 36 KM 45, 500 S/N EL PELIGRO",49-2883,29,RUTA SOL,"-58,14393954","-34,92052706",NaN
4,5,Buenos Aires,MARIO RAÚL SARASUA,492 Y 186 S/N COLONIA URQUIZA,491-4608,34,JOSE MELCHOR ROMERO,"-58,89381","-34,9444471",NaN


In [374]:
#elimino la columna 'col10'
clientes2.drop(columns='col10', inplace=True)

In [375]:
#renombro las columnas
clientes2 = clientes2.rename(columns={'ID':'IdCliente','X':'Longitud','Y':'Latitud'})

In [376]:
#normalizo los valores tipo string de dichas columnas
clientes2["Provincia"]= clientes2["Provincia"].str.title()
clientes2["Nombre_y_Apellido"]= clientes2["Nombre_y_Apellido"].str.title()
clientes2["Domicilio"]= clientes2["Domicilio"].str.title()
clientes2["Localidad"]= clientes2["Localidad"].str.title()

In [377]:
#donde tengo un NaN  le cambio el dato por 'Sin Dato'
clientes['Nombre_y_Apellido'].fillna('Sin Dato', inplace=True)
clientes['Provincia'].fillna('Sin Dato', inplace=True)
clientes['Domicilio'].fillna('Sin Dato', inplace=True)
clientes['Telefono'].fillna('Sin Dato', inplace=True)
clientes['Localidad'].fillna('Sin Dato', inplace=True)

In [378]:
# Modificamos el tipo de dato de la columna longitud
clientes2["Longitud"] = clientes2["Longitud"].str.replace(",", ".").astype(float)
# Modificamos el tipo de dato de la columna latitud
clientes2["Latitud"] = clientes2["Latitud"].str.replace(",", ".").astype(float)

In [379]:
clientes2['Longitud'].fillna('0.0', inplace=True)
clientes2['Latitud'].fillna('0.0', inplace=True)

In [380]:
#Normalizo provincia de Buenos Aires 
clientes2['Provincia']=clientes2.apply(lambda x: 'Buenos Aires' if x['Provincia']=='Ciudad de Buenos Aires'  else x['Provincia'], axis=1)

In [381]:
#Tratamiendo de acentos 
clientes2['Provincia']=[str(x).replace('á','a') for x in clientes2['Provincia']]
clientes2['Provincia']=[str(x).replace('é','e') for x in clientes2['Provincia']]
clientes2['Provincia']=[str(x).replace('í','i') for x in clientes2['Provincia']]
clientes2['Provincia']=[str(x).replace('ó','o') for x in clientes2['Provincia']]
clientes2['Provincia']=[str(x).replace('ú','u') for x in clientes2['Provincia']]
clientes2['Localidad']=[str(x).replace('á','a') for x in clientes2['Localidad']]
clientes2['Localidad']=[str(x).replace('é','e') for x in clientes2['Localidad']]
clientes2['Localidad']=[str(x).replace('í','i') for x in clientes2['Localidad']]
clientes2['Localidad']=[str(x).replace('ó','o') for x in clientes2['Localidad']]
clientes2['Localidad']=[str(x).replace('ú','u') for x in clientes2['Localidad']]

In [382]:
#Merge entre df Localidades y Clientes para traer dato IdLocalidad a Clientes
clientes2 = pd.merge(clientes2,localidades2, how="left", on=['Localidad','Provincia'])

In [383]:
#Reemplazo Latitud y Longitud sin datos con Latitud y Longitud de tabla Localidades 
clientes2["Longitud_x"]=clientes2.apply(lambda x: x["Longitud_y"] if x["Longitud_x"]==0  else x["Longitud_x"], axis=1)
clientes2["Latitud_x"]=clientes2.apply(lambda x: x["Latitud_y"] if x["Latitud_x"]==0  else x["Latitud_x"], axis=1)

In [384]:
#Renombro columnas
clientes2= clientes2.rename(columns= {"Longitud_x":"Longitud"})
clientes2= clientes2.rename(columns= {"Latitud_x":"Latitud"})

In [385]:
#Cambio de tipo de dato de Latitud y Longitud
clientes2["Latitud"]= clientes2.Latitud.astype(float)
clientes2["Longitud"]= clientes2.Longitud.astype(float)

In [386]:
clientes2.head()

,IdCliente,Provincia,Nombre_y_Apellido,Domicilio,Telefono,Edad,Localidad,Longitud,Latitud,index,Latitud_y,Longitud_y,IdDepartamento,IdLocalidad,IdProvincia
0,1,Buenos Aires,Heber Joni Santana,Las Heras Y Bat. 24 De Febrero 4150 Rincon De...,42-5161,58,Loma Verde,-58.818503,-34.309971,223.0,-34.328523,-58.850700,6252.0,6.252010e+09,6.0
1,2,Buenos Aires,Ana Sapriza,Pueyrredon Y Dupuy Ruta3 Km 52.500 S/N Bâº Lo...,49-7578,61,Santa Rosa,-58.730738,-34.939083,126.0,-34.961184,-58.730892,6134.0,6.134060e+09,6.0
2,3,Buenos Aires,Fernando Luis Saralegui,Calderon De La Barca 498,49-3435,15,Torres,-59.127941,-34.430822,505.0,-34.431822,-59.128774,6497.0,6.497090e+09,6.0
3,4,Buenos Aires,Manuela Sarasola,"Ruta 36 Km 45, 500 S/N El Peligro",49-2883,29,Ruta Sol,-58.143940,-34.920527,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Buenos Aires,Mario Raúl Sarasua,492 Y 186 S/N Colonia Urquiza,491-4608,34,Jose Melchor Romero,-58.893810,-34.944447,434.0,-34.945509,-58.036290,6441.0,6.441030e+09,6.0


In [387]:
#Elimino columnas que se agregaron en el merge
clientes2.drop(['index'], axis = 'columns', inplace=True)

In [388]:
#Funcion para encontrar Localidades por similitud - Se recupera IdLocalidad y IdProvincia
def verificarLocalidadClientes(row):
    ratiomayor=0
    for i in range(0,localidades2.shape[0]): 
            Ratio= fuzz.token_sort_ratio(row["Localidad"].lower(), localidades2.Localidad[i].lower())
            if Ratio>ratiomayor:
                ratiomayor=Ratio
                miloc=localidades2.Localidad[i]
                id=localidades2.IdLocalidad[i]
                idprov=localidades2.IdProvincia[i]

    row["Localidad"]=miloc
    row["IdLocalidad"]=id
    row["IdProvincia"]=idprov


    return row 

In [389]:
#Creo DF de clientes con IdLocalidad faltante 
clientes_nulos= clientes2.loc[clientes2['IdLocalidad'].isnull() == True].copy()

In [390]:
#Aplico funcion
clientes_nulos= clientes_nulos.apply(verificarLocalidadClientes,axis=1)

In [391]:
clientes_nulos.head()

,IdCliente,Provincia,Nombre_y_Apellido,Domicilio,Telefono,Edad,Localidad,Longitud,Latitud,Latitud_y,Longitud_y,IdDepartamento,IdLocalidad,IdProvincia
3,4,Buenos Aires,Manuela Sarasola,"Ruta 36 Km 45, 500 S/N El Peligro",49-2883,29,Barrio Ruta Sol,-58.143940,-34.920527,NaN,NaN,NaN,6441050000,6
5,6,Buenos Aires,Pedro Jesus Saravia,"Ruta 2 - Km 44, 500 S/N El Peligro",49-2350,18,Barrio Ruta Sol,-58.112264,-35.786216,NaN,NaN,NaN,6441050000,6
7,8,Buenos Aires,Carlos Jose Sarazola,Isla Santiago S/N Isla Santiago,623-9935,40,Isla Santiago (Oeste),-57.881542,-34.835031,NaN,NaN,NaN,6245010003,6
11,12,nan,Eduardo Vicente Sarni,Carlos Tejedor 444 El Pinar,4480-0769,52,Unanue,-58.579502,-34.827881,NaN,NaN,NaN,42154060000,42
12,13,Buenos Aires,Roberto Sartorio,Montevideo (Ruta 15) S/N Arroyo El Pescado,523-9987,41,El Puesto,-57.762213,-34.927584,NaN,NaN,NaN,10091060000,10


In [392]:
#Borro columnas innecesarias para el merge 
clientes_nulos.drop(columns=["Latitud","Longitud"],inplace=True)

In [393]:
#Merge de Clientes con DF Clientes con IDlocalidad Nulos (ya corregido)
clientes2 = pd.merge(clientes2,clientes_nulos, how="left", on=["IdCliente"])

In [394]:
#Completo IdLocalidad en tabla clientes2 original con 0 para prepararlos para el reemplazo
clientes2["IdLocalidad_x"]= clientes2["IdLocalidad_x"].fillna(0)
clientes2["IdProvincia_x"]= clientes2["IdProvincia_x"].fillna(0)

In [395]:
#Realizo reemplazo de IdLocalidad y IdProvincia traidos con el merge
clientes2["IdLocalidad_x"]=clientes2.apply(lambda x: x["IdLocalidad_y"] if x["IdLocalidad_x"]==0  else x["IdLocalidad_x"], axis=1)
clientes2["IdProvincia_x"]=clientes2.apply(lambda x: x["IdProvincia_y"] if x["IdProvincia_x"]==0  else x["IdProvincia_x"], axis=1)

In [396]:
#25 Localidades sin IdLocalidad - se lo completa como sin dato
clientes2["IdLocalidad_x"]=clientes2["IdLocalidad_x"].fillna(0)

In [397]:
clientes2.head()

,IdCliente,Provincia_x,Nombre_y_Apellido_x,Domicilio_x,Telefono_x,Edad_x,Localidad_x,Longitud,Latitud,Latitud_y_x,...,Nombre_y_Apellido_y,Domicilio_y,Telefono_y,Edad_y,Localidad_y,Latitud_y_y,Longitud_y_y,IdDepartamento_y,IdLocalidad_y,IdProvincia_y
0,1,Buenos Aires,Heber Joni Santana,Las Heras Y Bat. 24 De Febrero 4150 Rincon De...,42-5161,58,Loma Verde,-58.818503,-34.309971,-34.328523,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Buenos Aires,Ana Sapriza,Pueyrredon Y Dupuy Ruta3 Km 52.500 S/N Bâº Lo...,49-7578,61,Santa Rosa,-58.730738,-34.939083,-34.961184,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Buenos Aires,Fernando Luis Saralegui,Calderon De La Barca 498,49-3435,15,Torres,-59.127941,-34.430822,-34.431822,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Buenos Aires,Manuela Sarasola,"Ruta 36 Km 45, 500 S/N El Peligro",49-2883,29,Ruta Sol,-58.143940,-34.920527,NaN,...,Manuela Sarasola,"Ruta 36 Km 45, 500 S/N El Peligro",49-2883,29.0,Barrio Ruta Sol,NaN,NaN,NaN,6.441050e+09,6.0
4,5,Buenos Aires,Mario Raúl Sarasua,492 Y 186 S/N Colonia Urquiza,491-4608,34,Jose Melchor Romero,-58.893810,-34.944447,-34.945509,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [398]:
#Elimino columnas innecesarias
clientes2.drop(['Latitud_y_x','Longitud_y_x','IdDepartamento_x','Provincia_y','Localidad_y', 'Nombre_y_Apellido_y'], axis = 'columns', inplace=True)
clientes2.drop(['IdProvincia_y','IdLocalidad_y','IdDepartamento_y','Longitud_y_y','Latitud_y_y', 'Edad_y', 'Telefono_y', 'Domicilio_y'], axis = 'columns', inplace=True)

In [399]:
#Renombro columnas
clientes2= clientes2.rename(columns= {"Provincia_x":"Provincia"})
clientes2= clientes2.rename(columns= {"Nombre_y_Apellido_x":"Nombre_y_Apellido"})
clientes2= clientes2.rename(columns= {"Localidad_x":"Localidad"})
clientes2= clientes2.rename(columns= {"Longitud_x":"Longitud"})
clientes2= clientes2.rename(columns= {"Latitud_x":"Latitud"})
clientes2= clientes2.rename(columns= {"IdLocalidad_x":"IdLocalidad"})
clientes2= clientes2.rename(columns= {"IdProvincia_x":"IdProvincia"})

In [72]:
clientes2.shape

(735, 10)

# NORMALIZACION DE TABLA SUCURSALES

In [90]:
sucursales2 = sucursales.copy()

In [91]:
sucursales2.head()

,ID,Sucursal,Direccion,Localidad,Provincia,Latitud,Longitud
0,1,Cabildo,Av. Cabildo 1342,Ciudad de Buenos Aires,Ciudad de Buenos Aires,"-34,5678060","-58,4495720"
1,2,Palermo 1,Guatemala 5701,CABA,CABA,"-34,5790350","-58,4335660"
2,3,Palermo 2,Gral. Lucio Norberto Mansilla 2668,CABA,C deBuenos Aires,"-34,5959660","-58,4051500"
3,4,Corrientes,Av. Corrientes 2352,Ciudad de Buenos Aires,Bs As,"-34,6046850","-58,3987640"
4,5,Almagro,Venezuela 3650,Capital,Bs.As.,"-34,6173080","-58,4161790"


In [92]:
sucursales2.isnull().sum()

ID           0
Sucursal     0
Direccion    0
Localidad    0
Provincia    0
Latitud      0
Longitud     0
dtype: int64

In [93]:
#renombro las siguientes columnas
sucursales2.rename(columns={'ID':'IdSucursal'}, inplace=True)

In [94]:
#Tratamiendo de acentos 
sucursales2['Provincia']=[str(x).replace('á','a') for x in sucursales2['Provincia']]
sucursales2['Provincia']=[str(x).replace('é','e') for x in sucursales2['Provincia']]
sucursales2['Provincia']=[str(x).replace('í','i') for x in sucursales2['Provincia']]
sucursales2['Provincia']=[str(x).replace('ó','o') for x in sucursales2['Provincia']]
sucursales2['Provincia']=[str(x).replace('ú','u') for x in sucursales2['Provincia']]
sucursales2['Localidad']=[str(x).replace('á','a') for x in sucursales2['Localidad']]
sucursales2['Localidad']=[str(x).replace('é','e') for x in sucursales2['Localidad']]
sucursales2['Localidad']=[str(x).replace('í','i') for x in sucursales2['Localidad']]
sucursales2['Localidad']=[str(x).replace('ó','o') for x in sucursales2['Localidad']]
sucursales2['Localidad']=[str(x).replace('ú','u') for x in sucursales2['Localidad']]

In [95]:
#normalizo los valores tipo string de dichas columnas
sucursales2["Sucursal"]= sucursales2["Sucursal"].str.title()
sucursales2["Direccion"]= sucursales2["Direccion"].str.title()
sucursales2["Provincia"]= sucursales2["Provincia"].str.title()
sucursales2["Localidad"]= sucursales2["Localidad"].str.title()

In [96]:
# Modificamos el tipo de dato de la columna longitud
sucursales2["Longitud"] = sucursales2["Longitud"].str.replace(",", ".").astype(float)
# Modificamos el tipo de dato de la columna latitud
sucursales2["Latitud"] = sucursales2["Latitud"].str.replace(",", ".").astype(float)

In [97]:
sucursales2.isnull().sum()

IdSucursal    0
Sucursal      0
Direccion     0
Localidad     0
Provincia     0
Latitud       0
Longitud      0
dtype: int64

In [98]:
sucursales2.head()

,IdSucursal,Sucursal,Direccion,Localidad,Provincia,Latitud,Longitud
0,1,Cabildo,Av. Cabildo 1342,Ciudad De Buenos Aires,Ciudad De Buenos Aires,-34.567806,-58.449572
1,2,Palermo 1,Guatemala 5701,Caba,Caba,-34.579035,-58.433566
2,3,Palermo 2,Gral. Lucio Norberto Mansilla 2668,Caba,C Debuenos Aires,-34.595966,-58.405150
3,4,Corrientes,Av. Corrientes 2352,Ciudad De Buenos Aires,Bs As,-34.604685,-58.398764
4,5,Almagro,Venezuela 3650,Capital,Bs.As.,-34.617308,-58.416179


# NORMALIZACION DE LA TABLA PROVEEDORES

In [84]:
proveedores.head()

,IDProveedor,Nombre,Address,City,State,Country,departamen
0,1,NaN,BEREGUESTEIN 36,AVELLANEDA,BUENOS AIRES,ARGENTINA,AVELLANEDA
1,2,San Cirano,ALBERTI 965,CORONEL BRANDSEN,BUENOS AIRES,ARGENTINA,BRANDSEN
2,3,Bell S.A.,MORENO 331,CAMPANA,BUENOS AIRES,ARGENTINA,CAMPANA
3,4,Rivero Insumos,PTE. PERON 3872,VALENTIN ALSINA,BUENOS AIRES,ARGENTINA,LANUS
4,5,Laprida Computacion,AVDA. PEDRO PEREYRA 1339,LAPRIDA,BUENOS AIRES,ARGENTINA,LAPRIDA


In [85]:
proveedores.isnull().sum()

IDProveedor    0
Nombre         2
Address        0
City           0
State          0
Country        0
departamen     0
dtype: int64

In [86]:
#en la col nombre reemplazo los valores nulos por 'sin dato'
proveedores['Nombre'].fillna('Sin Dato', inplace=True)

In [87]:
#renombro las columnas
proveedores = proveedores.rename(columns={'Address':'Direccion', 'City': 'Ciudad', 'State':'Provincia', 'Country': 'Pais', 'departamen':'Localidad'})

In [88]:
#normalizo los valores de las columnas
proveedores['Nombre']=proveedores['Nombre'].str.title()
proveedores['Direccion']=proveedores['Direccion'].str.title()
proveedores['Ciudad']=proveedores['Ciudad'].str.title()
proveedores['Provincia']=proveedores['Provincia'].str.title()
proveedores['Pais']=proveedores['Pais'].str.title()
proveedores['Localidad']=proveedores['Localidad'].str.title()

In [89]:
proveedores.head()

,IDProveedor,Nombre,Direccion,Ciudad,Provincia,Pais,Localidad
0,1,Sin Dato,Bereguestein 36,Avellaneda,Buenos Aires,Argentina,Avellaneda
1,2,San Cirano,Alberti 965,Coronel Brandsen,Buenos Aires,Argentina,Brandsen
2,3,Bell S.A.,Moreno 331,Campana,Buenos Aires,Argentina,Campana
3,4,Rivero Insumos,Pte. Peron 3872,Valentin Alsina,Buenos Aires,Argentina,Lanus
4,5,Laprida Computacion,Avda. Pedro Pereyra 1339,Laprida,Buenos Aires,Argentina,Laprida
